# Tutorial 4: Unstructured Pruning on Bert

In [ ]:
!git clone https://github.com/tonytarizzo/mase.git
%cd mase
!python -m pip install -e . -vvv
%cd src

Pruning is a technique used to reduce the size and complexity of neural networks by removing unnecessary parameters (weights and connections) or structural components (neurons, filters, or layers). The goal is to create a smaller, more efficient model that maintains most of the original model's performance. The following benefits can be seen from pruning neural networks:

- **Reduce model size**: Deep neural networks often have millions of parameters, leading to large storage requirements.

- **Decrease inference time**: Fewer parameters mean fewer computations, resulting in faster predictions.

- **Improve generalization**: Removing unnecessary connections can help prevent overfitting.

- **Energy efficiency**: Smaller models require less energy to run, which is crucial for edge devices and mobile applications.

Structured pruning removes entire structures (e.g., channels, filters, or layers) from the network, while unstructured pruning removes individual weights or connections from the network, regardless of their location. In this tutorial, we'll build on top of Tutorial 3 by taking the quantized Bert model and running Mase's unstructured pruning pass. After pruning, we'll run further fine tuning iterations to retain sequence classification accuracy in the pruned model.

In [ ]:
checkpoint = "prajjwal1/bert-tiny"
tokenizer_checkpoint = "bert-base-uncased"
dataset_name = "imdb"

## Importing the model

If you are starting from scratch, you can create a MaseGraph for Bert by running the following cell.

In [ ]:
from transformers import AutoModelForSequenceClassification

from chop import MaseGraph
import chop.passes as passes

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.config.problem_type = "single_label_classification"

mg = MaseGraph(
    model,
    hf_input_names=[
        "input_ids",
        "attention_mask",
        "labels",
    ],
)

mg, _ = passes.init_metadata_analysis_pass(mg)
mg, _ = passes.add_common_metadata_analysis_pass(mg)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`past_key_values` were not specified as input names, but model.config.use_cache = True. Setting model.config.use_cache = False.
INFO     Getting dummy input for prajjwal1/bert-tiny.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tensor([[ 101, 9932, 2089, 2202, 2058, 1996, 2088, 2028, 2154,  102],
        [ 101, 2023, 2003, 2339, 2017, 2323, 4553, 4748, 4877,  102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[ 101, 9932, 2089, 2202, 2058, 1996, 2088, 2028, 2154,  102],
        [ 101, 2023, 2003, 2339, 2017, 2323, 4553, 4748, 4877,  102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]],


        [[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]]])
tensor([[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       

If you have previously ran the tutorial on Quantization-Aware Training (QAT), run the following cell to import the fine tuned checkpoint.

In [ ]:
from pathlib import Path
from chop import MaseGraph

mg = MaseGraph.from_checkpoint(f"{Path.home()}/tutorial_3_qat")

/content/mase/src/chop/ir/graph/mase_graph.py:380: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(f)


## Unstructured Pruning

Before running pruning, let's evaluate the model accuracy on the IMDb dataset. If you're coming from Tutorial, this would be the same as the accuracy after Quantization Aware Training (QAT). If you've just initialized the model, this will likely be a random guess (i.e. around 50%), in which case pruning wouldn't have a significant effect on the accuracy.

In [ ]:
from chop.tools import get_tokenized_dataset, get_trainer

dataset, tokenizer = get_tokenized_dataset(
    dataset=dataset_name,
    checkpoint=tokenizer_checkpoint,
    return_tokenizer=True,
)

trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric="accuracy",
)

# Evaluate accuracy
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

INFO     Tokenizing dataset imdb with AutoTokenizer for bert-base-uncased.


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/content/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation accuracy: 0.84472


To run the pruning pass, we pass the following pruning configuration dictionary, which defines the following parameters.

- **Sparsity**: a value between 0 and 1, expressing the proportion of elements in the model that should be pruned (i.e. set to 0).

- **Method**: several pruning methods are supported, including ``Random`` and ``L1-Norm``.

- **Scope**: defines whether to consider each weight/activation tensor individually (``local``) or all tensors in the model (``global``) when obtaining statistics for pruning (e.g. absolute value threshold for pruning)

We'll start by running random pruning with local scope, at a fixed sparsity. This may be suboptimal, but in future tutorials we'll see how to find optimal pruning and quantization configurations for a given model on a specified dataset.

In [ ]:
import chop.passes as passes

pruning_config = {
    "weight": {
        "sparsity": 0.2,
        "method": "l1-norm",
        "scope": "local",
    },
    "activation": {
        "sparsity": 0.2,
        "method": "l1-norm",
        "scope": "local",
    },
}

mg, _ = passes.prune_transform_pass(mg, pass_args=pruning_config)

INFO     Pruning module: bert_encoder_layer_0_attention_self_query
INFO     Pruning module: bert_encoder_layer_0_attention_self_key
INFO     Pruning module: bert_encoder_layer_0_attention_self_value
INFO     Pruning module: bert_encoder_layer_0_attention_output_dense
INFO     Pruning module: bert_encoder_layer_0_intermediate_dense
INFO     Pruning module: bert_encoder_layer_0_output_dense
INFO     Pruning module: bert_encoder_layer_1_attention_self_query
INFO     Pruning module: bert_encoder_layer_1_attention_self_key
INFO     Pruning module: bert_encoder_layer_1_attention_self_value
INFO     Pruning module: bert_encoder_layer_1_attention_output_dense
INFO     Pruning module: bert_encoder_layer_1_intermediate_dense
INFO     Pruning module: bert_encoder_layer_1_output_dense
INFO     Pruning module: bert_pooler_dense
INFO     Pruning module: classifier


Let's evaluate the effect of pruning on accuracy. It's likely to observe drops of around 10% or more.

In [ ]:
trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric="accuracy",
    num_train_epochs=5,
)

# Evaluate accuracy
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

/content/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation accuracy: 0.8402


To overcome the drop in accuracy, we'll run a few finetuning epochs. This allows the model to adapt to the new pruning mask.

In [ ]:
trainer.train()

Step,Training Loss
500,0.384800
1000,0.382400
1500,0.395400
2000,0.377300
2500,0.377600
3000,0.394400
3500,0.389100
4000,0.375000
4500,0.350400
5000,0.360100


TrainOutput(global_step=15625, training_loss=0.3591796530761719, metrics={'train_runtime': 369.4186, 'train_samples_per_second': 338.37, 'train_steps_per_second': 42.296, 'total_flos': 0.0, 'train_loss': 0.3591796530761719, 'epoch': 5.0})

Let's evaluate the model accuracy after finetuning. We should see that the accuracy is reverted back to the original level.

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

Evaluation accuracy: 0.85688


In [ ]:
### Full combined code for Lab 1
from transformers import AutoModelForSequenceClassification
from chop import MaseGraph
import chop.passes as passes
from pathlib import Path
from chop.tools import get_tokenized_dataset, get_trainer

sparsity_levels = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
methods = ["l1-norm", "random"]

results = [
    {
        "sparsity": sparsity,
        "l1_initial_pruned_accuracy": None,
        "l1_fine_tuned_accuracy": None,
        "rand_initial_pruned_accuracy": None,
        "rand_fine_tuned_accuracy": None,
    }
    for sparsity in sparsity_levels
]

for iter, sparsity in enumerate(sparsity_levels):
    for method in methods:
        checkpoint = "prajjwal1/bert-tiny"
        tokenizer_checkpoint = "bert-base-uncased"
        dataset_name = "imdb"

        model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
        model.config.problem_type = "single_label_classification"

        mg = MaseGraph(
            model,
            hf_input_names=[
                "input_ids",
                "attention_mask",
                "labels",
            ],
        )

        mg, _ = passes.init_metadata_analysis_pass(mg)
        mg, _ = passes.add_common_metadata_analysis_pass(mg)

        mg = MaseGraph.from_checkpoint(f"{Path.home()}/tutorial_3_qat")

        dataset, tokenizer = get_tokenized_dataset(
            dataset=dataset_name,
            checkpoint=tokenizer_checkpoint,
            return_tokenizer=True,
        )

        trainer = get_trainer(
            model=mg.model,
            tokenized_dataset=dataset,
            tokenizer=tokenizer,
            evaluate_metric="accuracy",
        )

        # Evaluate initial accuracy
        eval_results = trainer.evaluate()
        initial_accuracy = eval_results["eval_accuracy"]
        print(f"Initial accuracy (method={method}, sparsity={sparsity}): {initial_accuracy}")

        # Apply pruning
        pruning_config = {
            "weight": {
                "sparsity": sparsity,
                "method": method,
                "scope": "local",
            },
            "activation": {
                "sparsity": sparsity,
                "method": method,
                "scope": "local",
            },
        }

        mg, _ = passes.prune_transform_pass(mg, pass_args=pruning_config)

        trainer = get_trainer(
            model=mg.model,
            tokenized_dataset=dataset,
            tokenizer=tokenizer,
            evaluate_metric="accuracy",
            num_train_epochs=5,
        )

        # Evaluate pruned accuracy
        eval_results = trainer.evaluate()
        pruned_accuracy = eval_results["eval_accuracy"]
        print(f"Pruned accuracy (method={method}, sparsity={sparsity}): {pruned_accuracy}")

        # Fine-tune and evaluate
        trainer.train()
        eval_results = trainer.evaluate()
        fine_tuned_accuracy = eval_results["eval_accuracy"]
        print(f"Fine-tuned accuracy (method={method}, sparsity={sparsity}): {fine_tuned_accuracy}")

        # Update results
        for result in results:
            if result["sparsity"] == sparsity:
                if method == "l1-norm":
                    result["l1_initial_pruned_accuracy"] = pruned_accuracy
                    result["l1_fine_tuned_accuracy"] = fine_tuned_accuracy
                elif method == "random":
                    result["rand_initial_pruned_accuracy"] = pruned_accuracy
                    result["rand_fine_tuned_accuracy"] = fine_tuned_accuracy

for result in results:
    print(result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`past_key_values` were not specified as input names, but model.config.use_cache = True. Setting model.config.use_cache = False.
INFO     Getting dummy input for prajjwal1/bert-tiny.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Initial accuracy (method=random, sparsity=0.9): 0.84468


/content/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Pruned accuracy (method=random, sparsity=0.9): 0.50224


Step,Training Loss
500,0.693100
1000,0.693200
1500,0.693100
2000,0.693200
2500,0.693100
3000,0.693200
3500,0.693100
4000,0.693100
4500,0.693200
5000,0.693100


Fine-tuned accuracy (method=random, sparsity=0.9): 0.5034
{'sparsity': 0.2, 'l1_initial_pruned_accuracy': 0.83992, 'l1_fine_tuned_accuracy': 0.85832, 'rand_initial_pruned_accuracy': 0.6308, 'rand_fine_tuned_accuracy': 0.8232}
{'sparsity': 0.3, 'l1_initial_pruned_accuracy': 0.83004, 'l1_fine_tuned_accuracy': 0.85504, 'rand_initial_pruned_accuracy': 0.54652, 'rand_fine_tuned_accuracy': 0.80812}
{'sparsity': 0.4, 'l1_initial_pruned_accuracy': 0.80796, 'l1_fine_tuned_accuracy': 0.8468, 'rand_initial_pruned_accuracy': 0.50972, 'rand_fine_tuned_accuracy': 0.78696}
{'sparsity': 0.5, 'l1_initial_pruned_accuracy': 0.76728, 'l1_fine_tuned_accuracy': 0.84148, 'rand_initial_pruned_accuracy': 0.5066, 'rand_fine_tuned_accuracy': 0.74436}
{'sparsity': 0.6, 'l1_initial_pruned_accuracy': 0.563, 'l1_fine_tuned_accuracy': 0.82284, 'rand_initial_pruned_accuracy': 0.5046, 'rand_fine_tuned_accuracy': 0.52732}
{'sparsity': 0.7, 'l1_initial_pruned_accuracy': 0.54052, 'l1_fine_tuned_accuracy': 0.8098, 'rand_in

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Printed results were imported here
results = [
    {
        "sparsity": 0.1,
        "l1_initial_pruned_accuracy": 0.84368,
        "l1_fine_tuned_accuracy": 0.8604,
        "rand_initial_pruned_accuracy": 0.73676,
        "rand_fine_tuned_accuracy": 0.83952
    },
    {
        "sparsity": 0.2,
        "l1_initial_pruned_accuracy": 0.83992,
        "l1_fine_tuned_accuracy": 0.85832,
        "rand_initial_pruned_accuracy": 0.6308,
        "rand_fine_tuned_accuracy": 0.8232
    },
    {
        "sparsity": 0.3,
        "l1_initial_pruned_accuracy": 0.83004,
        "l1_fine_tuned_accuracy": 0.85504,
        "rand_initial_pruned_accuracy": 0.54652,
        "rand_fine_tuned_accuracy": 0.80812
    },
    {
        "sparsity": 0.4,
        "l1_initial_pruned_accuracy": 0.80796,
        "l1_fine_tuned_accuracy": 0.8468,
        "rand_initial_pruned_accuracy": 0.50972,
        "rand_fine_tuned_accuracy": 0.78696
    },
    {
        "sparsity": 0.5,
        "l1_initial_pruned_accuracy": 0.76728,
        "l1_fine_tuned_accuracy": 0.84148,
        "rand_initial_pruned_accuracy": 0.5066,
        "rand_fine_tuned_accuracy": 0.74436
    },
    {
        "sparsity": 0.6,
        "l1_initial_pruned_accuracy": 0.563,
        "l1_fine_tuned_accuracy": 0.82284,
        "rand_initial_pruned_accuracy": 0.5046,
        "rand_fine_tuned_accuracy": 0.52732
    },
    {
        "sparsity": 0.7,
        "l1_initial_pruned_accuracy": 0.54052,
        "l1_fine_tuned_accuracy": 0.8098,
        "rand_initial_pruned_accuracy": 0.50292,
        "rand_fine_tuned_accuracy": 0.50436
    },
    {
        "sparsity": 0.8,
        "l1_initial_pruned_accuracy": 0.50076,
        "l1_fine_tuned_accuracy": 0.76204,
        "rand_initial_pruned_accuracy": 0.49896,
        "rand_fine_tuned_accuracy": 0.504
    },
    {
        "sparsity": 0.9,
        "l1_initial_pruned_accuracy": 0.50312,
        "l1_fine_tuned_accuracy": 0.56104,
        "rand_initial_pruned_accuracy": 0.50224,
        "rand_fine_tuned_accuracy": 0.5034
    }
]

def plot_pruning_results_separated(results):
    # Extract sparsity levels
    sparsities = [res["sparsity"] for res in results]

    # Extract accuracies
    l1_initial = [res["l1_initial_pruned_accuracy"] for res in results]
    l1_fine_tuned = [res["l1_fine_tuned_accuracy"] for res in results]
    rand_initial = [res["rand_initial_pruned_accuracy"] for res in results]
    rand_fine_tuned = [res["rand_fine_tuned_accuracy"] for res in results]

    # Calculate highest achieved accuracy per sparsity
    highest_accuracies = [
        max(filter(None, [l1_init, l1_fine, rand_init, rand_fine]))
        for l1_init, l1_fine, rand_init, rand_fine in zip(l1_initial, l1_fine_tuned, rand_initial, rand_fine_tuned)
    ]

    # Plot 1: Highest achieved accuracy per sparsity
    plt.figure(figsize=(10, 6))
    plt.plot(
        sparsities,
        highest_accuracies,
        marker="o",
        linestyle="--",
        linewidth=2,
        label="Highest Accuracy",
    )
    plt.grid(visible=True, linestyle="--", alpha=0.5)
    plt.xlabel("Sparsity", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)
    plt.title("Highest Achieved Accuracy by Sparsity", fontsize=14, pad=15)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x*100:.0f}%"))
    for x, y in zip(sparsities, highest_accuracies):
        plt.text(x, y + 0.005, f"{y*100:.1f}%", fontsize=8, ha="center")
    plt.tight_layout()
    plt.savefig("highest_accuracy_by_sparsity.png", dpi=300)
    plt.show()

    # Plot 2: Random and L1-Norm accuracy curves
    plt.figure(figsize=(10, 6))
    rand_accuracies = [
        max(filter(None, [rand_init, rand_fine]))
        for rand_init, rand_fine in zip(rand_initial, rand_fine_tuned)
    ]
    l1_accuracies = [
        max(filter(None, [l1_init, l1_fine]))
        for l1_init, l1_fine in zip(l1_initial, l1_fine_tuned)
    ]
    plt.plot(
        sparsities,
        rand_accuracies,
        marker="s",
        linestyle="-.",
        linewidth=2,
        label="Random Pruning",
    )
    plt.plot(
        sparsities,
        l1_accuracies,
        marker="d",
        linestyle=":",
        linewidth=2,
        label="L1-Norm Pruning",
    )
    plt.grid(visible=True, linestyle="--", alpha=0.5)
    plt.xlabel("Sparsity", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)
    plt.title("Effect of Sparsity on Pruning Accuracy", fontsize=14, pad=15)
    plt.legend(fontsize=10, loc="lower left")
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x*100:.0f}%"))
    for x, y in zip(sparsities, rand_accuracies):
        plt.text(x, y + 0.005, f"{y*100:.1f}%", fontsize=8, ha="center")
    for x, y in zip(sparsities, l1_accuracies):
        plt.text(x, y + 0.005, f"{y*100:.1f}%", fontsize=8, ha="center")
    plt.tight_layout()
    plt.savefig("pruning_accuracy_by_sparsity.png", dpi=300)
    plt.show()

# Call the function
plot_pruning_results_separated(results)